In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# This notebook is largely based on that from https://github.com/jarokaz/mlops-labs
# TODO: Update the kfp to latest version

# Orchestrating AutoML Tables training and deployment with Kubeflow Pipelines

In this lab you develop a continous training and deployment pipeline using Kubeflow Pipelines, BigQuery and AutoML Tables.

The scenario used in the lab is  predicting customer lifetime value (CLV).

The goal of CLV modeling is to identify the most valuable customers - customers that are going to generate the highest value in a given future time range. The CLV models are built from a variety of data sources - historical sales data being the most important one and in many cases the only one. 

Predicting Customer Lifetime Value (CLV)  is a representative example of a use case where you may need to fine-tune and re-train a predictive model on a frequent basis. As there is a constant flow of new sales transactions that constitute the core of training data, models have to be kept up to date with evolving purchase patterns. Automation of model training and deployment is critical. 

In the CLV model developed in this lab, the historical sales transactions are preprocessed and aggregated to engineer a set of latent features  representing the so-called RFM characteristics of your customers:
- Recency: How active have they been recently?
- Frequency: How often do they buy?
- Monetary: What amount do they spend?

The following diagram shows a succession of past sales for a set of four customers.

![clv_timeline](../images/clv-timeline.png)

The diagram illustrates the RFM values for the customers, showing for each customer:
- Recency: The time between the last purchase and today, represented by the distance between the leftmost circle and the vertical dotted line that's labeled **Now**.
- Frequency: The time between purchaes, represented by the distance between the circles on a single line.
- Monetary: The amount of money spent on each purchase, represented by the size of the circle.

As demonstrated in the lab you usually create multiple features per each characteristic. For example, in the lab, Recency is captured by two features: *recency* and *T*.

The RFM input features and the target label are engineered using the following process:
- A time series of of historical sales transactions for a given customer is divided into two time periods: *the features period* and *the predict period*. A point in time that is used to divide the time series is referred two as *the threshold date*. 
- The transactions in *the features period* are aggregated to create the latent RFM input features 
- The transactions in *the predict period* are aggregated to calculate the target label representing the expected total value of the customer

This process results in a single example per customer and a set of examples across a customer population constitutes a training set.

The pipeline implemented in the lab, uses BigQuery as a source of historical sales transactions. BigQuery is also used to engineer RFM features. The model is then trained and deployed using AutoML Tables. The below diagram represents the control and data flow implemented by the pipeline.


![Training pipeline](../images/clv_train.png)

1. The BQ query is run to process sales transactions in the *transactions* table into RFM features in the *features* table. 
1. The data from the *features* table is imported into the AutoML dataset
1. The AutoML model is trained on the imported dataset
1. After the training completes the evaluation metrics are retrieved and compared against the performance threshold
1. If the model performs better than the threshold the model is deployed to AutoML Deployment

The sample dataset used in the lab is based on the publicaly available [Online Retail Data Set](http://archive.ics.uci.edu/ml/datasets/Online+Retail) from the UCI Machine Learning Repository. The original dataset was preprocessed to conform to the following schema:

| Field | Type | Description |
|-------|------|-------------|
| customer_id | string | A unique customer ID |
| order_date | date (yyyy-MM-dd) | The date of a transaction. Transactions (potentially from multiple invoices) are grouped by day |
| quantity | integer | A number of items of a single SKU in a transaction |
| unit_price | float | A unit price of a SKU |

The feature engineering query generates the features table with the below schema. 


| Field | Type | Description |
|-------|------|-------------|
| monetary | Float | The total spend by a customer in the features period|
| frequency | Integer | The number of transactions placed by a customer in the features period |
| recency | Integer |  The time (in days) between the first and the last orders in the features period |
| T | Integer | The time between the first order placed and in the threshold date|
| time_between | Float |  The average time betwee orders in the features period |
| avg_basket_value | Float |  The averate monetary value of the customer's basket in the features period |
| avg_basket_size | Float |  The average number of items in a basket in the features perio|
| cnt_returns | Integer |  The number of returns in the features period|
| target_monetary | Float | The total amount spent in the predict period. This is the label for predictions|





## Prepare lab environment
Let's start with configuring your GCP environment settings and uploading the sales transactions into BigQuery.

In [ ]:
from typing import NamedTuple

import kfp
import kfp.components as comp

from google.cloud import bigquery

from jinja2 import Template

%load_ext google.cloud.bigquery

In [ ]:
import os
# Service account to access the Kubeflow pipeline service
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './config/kubeflow-pipeline-fantasy.json'

In [ ]:
# ! gcloud services enable automl.googleapis.com

In [ ]:
# Add Automl permission to "Service Account used for Kubeflow user actions."

### Set lab settings
Make sure to update the following values with you environment settings.

In [ ]:
PROJECT_ID = 'kubeflow-pipeline-fantasy'
GCS_BUCKET='gs://kubeflow-pipeline-ui'

In [ ]:
DATASET_LOCATION = 'US'
DATASET_ID = 'lab_automl'
TRANSACTIONS_TABLE_ID = 'transactions'
TRANSACTIONS_TABLE_SCHEMA = 'customer_id:STRING,order_date:DATE,quantity:INTEGER,unit_price:FLOAT'
# This link is public accessable
TRANSACTIONS_SOURCE_FILE='gs://kubeflow-pipeline-ui/cl_data/transactions.csv'
COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.1.36/components/gcp/'

In [ ]:
# Optional Parameters, but required for running outside Kubeflow cluster

# # The host for full deployment of Kubeflow ends with '/pipeline'
# HOST = ''
# # Full deployment of Kubeflow on GCP is usually protected through IAP, therefore the following 
# # will be needed to access the endpoint
# CLIENT_ID = ''
# OTHER_CLIENT_ID = ''
# OTHER_CLIENT_SECRET = ''

# The host for managed 'AI Platform Pipeline' ends with 'pipelines.googleusercontent.com'
HOST = 'https://69a95965149a4145-dot-asia-east1.pipelines.googleusercontent.com'

In [ ]:
# This is to ensure the proper access token is present to reach the end point for managed 'AI Platform Pipeline'
# If you are not working with managed 'AI Platform Pipeline', this step is not necessary
! gcloud auth print-access-token

In [ ]:
# Create kfp client
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    if HOST.endswith('googleusercontent.com'):
        CLIENT_ID = None
        OTHER_CLIENT_ID = None
        OTHER_CLIENT_SECRET = None

    client = kfp.Client(host=HOST, 
                        client_id=CLIENT_ID,
                        other_client_id=OTHER_CLIENT_ID, 
                        other_client_secret=OTHER_CLIENT_SECRET)

### Create a BigQuery dataset

In [ ]:
!bq --location=$DATASET_LOCATION --project_id=$PROJECT_ID mk --dataset $DATASET_ID

### Load sale transactions data to BigQuery

In [ ]:
!bq --project_id=$PROJECT_ID --dataset_id=$DATASET_ID load \
--source_format=CSV \
--skip_leading_rows=1 \
--replace \
$TRANSACTIONS_TABLE_ID \
$TRANSACTIONS_SOURCE_FILE \
$TRANSACTIONS_TABLE_SCHEMA

### Explore the dataset
To query data in BigQuery you can use BigQuery Python client library ....

In [ ]:
bq_client = bigquery.Client()

In [ ]:
query_template = """
SELECT *
FROM `{{ source_table }}`
LIMIT 100
"""

query = Template(query_template).render(
    source_table='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TRANSACTIONS_TABLE_ID))

df = bq_client.query(query).to_dataframe()
df.head(10)

... or Jupyter the `%%bigquery` magic

In [ ]:
%%bigquery --project $PROJECT_ID
SELECT *
FROM `lab_automl.transactions`
WHERE customer_id='16553'

As you can see there are multiple sales transactions per customer. They represent the purchasing history and behavior of a given customer. For example, 
the customer identified by 16553 has 85 orders. Most of them are new purchases. Some of them are returns - the records with a negative quantity.

The feature engineering query converts these 85 records into a single record representing the RFM charateristics of this customer.

## Create the KFP training pipeline

### Create component factories for the pre-defined GCP components

In [ ]:
component_store = kfp.components.ComponentStore(
    local_search_paths=None,
    url_search_prefixes=[COMPONENT_URL_SEARCH_PREFIX])
    
automl_create_dataset_op = component_store.load_component('automl/create_dataset_for_tables')
automl_import_data_from_bq_op = component_store.load_component('automl/import_data_from_bigquery')
automl_create_model__op = component_store.load_component('automl/create_model_for_tables')
automl_split_dataset_table_column_names_op = component_store.load_component('automl/split_dataset_table_column_names')

### Create custom components

#### Create a base docker image for the custom components

In [ ]:
%%writefile Dockerfile
FROM python:3.7
RUN pip3 install google-cloud-bigquery==1.24.0 google-api-core==1.16.0 google-cloud-automl==0.10.0 grpcio==1.27.2 google-cloud-core==1.3.0

In [ ]:
IMAGE_NAME="lab_automl_components"
IMAGE_URI="gcr.io/{}/{}:latest".format(PROJECT_ID, IMAGE_NAME)

In [ ]:
import subprocess
# ! gcloud builds submit --tag ${IMAGE_NAME} ${APP_FOLDER}
cmd = ['gcloud', 'builds', 'submit', '--tag', IMAGE_URI, '.']
build_log = (subprocess.run(cmd, stdout=subprocess.PIPE).stdout[:-1].decode('utf-8'))
print(build_log)

import re
m = re.search(r'latest: digest: sha256:.* size', build_log)
digest = m.group(0).split(' ')[2]

image_full_uri = "gcr.io/{}/{}@{}".format(PROJECT_ID, IMAGE_NAME, digest)

#### Create BQ query component

In [ ]:
def bq_query(query: str, 
             project_id:str, 
             dataset_id: str, 
             table_id: str, 
             location: str) -> NamedTuple('Outputs', [('table_uri', str), ('job_id', str)]):

    from google.cloud import bigquery
    from google.api_core import exceptions
    import logging
    import os
    import uuid
    
    logging.basicConfig(level=logging.INFO)
    
    client = bigquery.Client(project=project_id, location=location)
    
    job_config = bigquery.QueryJobConfig()
    job_config.create_disposition = bigquery.job.CreateDisposition.CREATE_IF_NEEDED
    job_config.write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
    job_id = 'query_' + os.environ.get('KFP_POD_NAME', uuid.uuid1().hex)
    
    dataset_ref = client.dataset(dataset_id)
    try:
        dataset = client.get_dataset(dataset_ref)
    except exceptions.NotFound:
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = location
        logging.info('Creating dataset {}'.format(dataset_id))
        client.create_dataset(dataset)
     
    table_id = table_id if table_id else job_id
    table_ref = dataset_ref.table(table_id)
    job_config.destination = table_ref
    logging.info('Submitting the job {}'.format(job_id))
    query_job = client.query(query, job_config, job_id=job_id)
    query_job.result() # Wait for query to finish
            
    table_uri = 'bq://{}.{}.{}'.format(project_id, dataset_id, table_id)
    
    return (table_uri, job_id)

In [ ]:
bq_query_op = comp.func_to_container_op(bq_query, base_image=image_full_uri)

#### Create a component that retrieves and logs AutoML evaluation metrics

In [ ]:
def automl_log_regression_metrics(
    model_path: str,
    primary_metric:str) -> NamedTuple('Outputs', [('primary_metric_value', float), 
                                                  ('mlpipeline_metrics', 'Metrics')]):
    
    import logging
    import json
    from google.cloud import automl_v1beta1 as automl
    from collections import namedtuple

    logging.basicConfig(level=logging.INFO)
    client = automl.TablesClient()

    # Retrieve evaluation metrics
    for evaluation in client.list_model_evaluations(model_name=model_path):
        if evaluation.regression_evaluation_metrics.ListFields():
            evaluation_metrics = evaluation.regression_evaluation_metrics      
    primary_metric_value = getattr(evaluation_metrics, primary_metric)
    
    # Write the primary metric as a KFP pipeline metric
    metrics = {
        'metrics': [{
            'name': primary_metric.replace('_', '-'),
            'numberValue': primary_metric_value
        }]
    }
    divmod_output = namedtuple('AutoMLMetricsOutput', ['primary_metric_value', 'mlpipeline_metrics'])
    return divmod_output(primary_metric_value, json.dumps(metrics))

In [ ]:
log_regression_metrics_op = comp.func_to_container_op(automl_log_regression_metrics, base_image=image_full_uri)

#### Create a component that deploys an AutoML model

In [ ]:
def automl_deploy_model(model_path: str):
    
    import logging
    from google.cloud import automl_v1beta1 as automl
    from google.cloud.automl_v1beta1 import enums
    
    logging.basicConfig(level=logging.INFO)
    client = automl.TablesClient()
    
    model = client.get_model(model_name=model_path)
    if model.deployment_state != enums.Model.DeploymentState.DEPLOYED:
        logging.info("Starting model deployment: {}".format(model_path))
        response = client.deploy_model(model_name=model_path)
        response.result() # Wait for operation to complete
        logging.info("Deployment completed")
    else:
         logging.info("Model already deployed")
    
    
deploy_model_op = comp.func_to_container_op(automl_deploy_model, base_image=image_full_uri)

### Define the pipeline

#### Define the pipeline function

In [ ]:
@kfp.dsl.pipeline(
    name='CLV Training',
    description='CLV Training Pipeline using BigQuery for feature engineering and Automl Tables for model training'
)
def clv_train(
    project_id: str,
    feature_engineering_query: str,
    features_dataset_id: str,
    features_dataset_location: str,    
    features_table_id: str ='features',
    aml_compute_region: str ='us-central1',
    aml_dataset_name: str ='clv_features',
    target_column_name: str ='target_monetary',
    aml_model_name: str ='clv_regression',
    train_budget: 'Integer' =1000,
    optimization_objective: str ='MINIMIZE_MAE',
    primary_metric: str ='mean_absolute_error',
    deployment_threshold: 'Float' ='900'
    ):
    """Trains a Customer Lifetime Value model"""
    
    # Use BigQuery to engineer features from transaction data
    engineer_features = bq_query_op(
        query=feature_engineering_query,
        project_id=project_id,
        dataset_id=features_dataset_id,
        table_id=features_table_id,
        location=features_dataset_location)
    
    # Create an AML Dataset
    create_dataset = automl_create_dataset_op(
        gcp_project_id=project_id,
        gcp_region=aml_compute_region,
        display_name=aml_dataset_name
    )
    
    # Import the features from BigQuery to AML Dataset
    import_data = automl_import_data_from_bq_op(
        dataset_path=create_dataset.outputs['dataset_path'],
        input_uri=engineer_features.outputs['table_uri']
    )
    
    # Set the target and feature columns
    split_column_specs = automl_split_dataset_table_column_names_op(
        dataset_path=import_data.outputs['dataset_path'],
        table_index=0,
        target_column_name=target_column_name
    )
    
    # Create a model
    create_model = automl_create_model__op(
        gcp_project_id=project_id,
        gcp_region=aml_compute_region,
        display_name=aml_model_name,
        dataset_id=create_dataset.outputs['dataset_id'],
        target_column_path=split_column_specs.outputs['target_column_path'],
        input_feature_column_paths=split_column_specs.outputs['feature_column_paths'],
        optimization_objective=optimization_objective,
        train_budget_milli_node_hours=train_budget
    )
    
    # Retrieve the primary metric from the model evaluations
    log_regression_metrics = log_regression_metrics_op(create_model.outputs['model_path'], primary_metric)
    
    # Deploy the model if the primary metric is better than threshold
    with kfp.dsl.Condition(log_regression_metrics.outputs['primary_metric_value'] < deployment_threshold):
        deploy_model = deploy_model_op(create_model.outputs['model_path'])
    
    kfp.dsl.get_pipeline_conf()

In [ ]:
query_template = '''
WITH
  order_summaries as (
    SELECT
      a.customer_id,
      a.order_date,
      a.order_value,
      a.order_qty_articles
    FROM
    (
      SELECT
        customer_id,
        order_date,
        ROUND(SUM(unit_price * quantity), 2) AS order_value,
        SUM(quantity) AS order_qty_articles,
        (
          SELECT
            MAX(order_date)
          FROM
            `{{ data_source_id }}` tl
          WHERE
            tl.customer_id = t.customer_id
        ) latest_order
      FROM
        `{{ data_source_id }}` t
      GROUP BY
          customer_id,
          order_date
    ) a

    INNER JOIN (
      -- Only customers with more than one positive order values before threshold.
      SELECT
        customer_id
      FROM (
        -- Customers and how many positive order values  before threshold.
        SELECT
          customer_id,
          SUM(positive_value) cnt_positive_value
        FROM (
          -- Customer with whether order was positive or not at each date.
          SELECT
            customer_id,
            (
              CASE
                WHEN SUM(unit_price * quantity) > 0 THEN 1
                ELSE 0
              END ) positive_value
          FROM
            `{{ data_source_id }}`
          WHERE
            order_date < DATE("{{ threshold_date }}")
          GROUP BY
            customer_id,
            order_date)
        GROUP BY
          customer_id )
      WHERE
        cnt_positive_value > 1
      ) b
    ON
      a.customer_id = b.customer_id
    --[START common_clean]
    WHERE
      -- Bought in the past 3 months
      DATE_DIFF(DATE("{{ predict_end }}"), latest_order, DAY) <= 90
      -- Make sure returns are consistent.
      AND (
        (order_qty_articles > 0 and order_Value > 0) OR
        (order_qty_articles < 0 and order_Value < 0)
      ))
          
SELECT
--  tf.customer_id,
  ROUND(tf.monetary, 2) as monetary,
  tf.cnt_orders AS frequency,
  tf.recency,
  tf.T,
  ROUND(tf.recency/cnt_orders, 2) AS time_between,
  ROUND(tf.avg_basket_value, 2) AS avg_basket_value,
  ROUND(tf.avg_basket_size, 2) AS avg_basket_size,
  tf.cnt_returns,
  -- Target calculated for overall period
  ROUND(tt.target_monetary, 2) as target_monetary
FROM
  -- This SELECT uses only data before threshold to make features.
  (
    SELECT
      customer_id,
      SUM(order_value) AS monetary,
      DATE_DIFF(MAX(order_date), MIN(order_date), DAY) AS recency,
      DATE_DIFF(DATE('{{ threshold_date }}'), MIN(order_date), DAY) AS T,
      COUNT(DISTINCT order_date) AS cnt_orders,
      AVG(order_qty_articles) avg_basket_size,
      AVG(order_value) avg_basket_value,
      SUM(CASE
          WHEN order_value < 1 THEN 1
          ELSE 0 END) AS cnt_returns
    FROM
      order_summaries a
    WHERE
      order_date <= DATE('{{ threshold_date }}')
    GROUP BY
      customer_id) tf,

  -- This SELECT uses data after threshold to calculate the target )
  (
    SELECT
      customer_id,
      SUM(order_value) target_monetary
    FROM
      order_summaries
      WHERE order_date > DATE('{{ threshold_date }}')
    GROUP BY
      customer_id) tt
WHERE
  tf.customer_id = tt.customer_id
  AND tf.monetary > 0
  AND tf.monetary <= {{ max_monetary }}
'''
    
query = Template(query_template).render(
    data_source_id='{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TRANSACTIONS_TABLE_ID),
    threshold_date='2011-08-08',
    predict_end='2011-12-12',
    max_monetary=15000)

In [ ]:
pipeline_func = clv_train

In [ ]:
experiment_name = 'clv_kubeflow'

arguments = {
    'project_id': PROJECT_ID,
    'features_dataset_id': DATASET_ID,
    'features_dataset_location':  DATASET_LOCATION,
    'feature_engineering_query': query}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)